Objectif : utiliser des random forests afin de prédire l'évolution de l'espèce en fonction du changement climatique dans le temps et l'espace

In [3]:
import pandas as pd
import numpy as np
import xarray as xr
from netCDF4 import Dataset
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import s3fs 

1. retrieving data from the MinIO Client cloud 

Data on Atlantic puffins : hvf
Data on climate change  

In [25]:
import xarray as xr

print("Available engines in xarray:", xr.backends.list_engines())



Available engines in xarray: {'scipy': <ScipyBackendEntrypoint>
  Open netCDF files (.nc, .nc4, .cdf and .gz) using scipy in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.ScipyBackendEntrypoint.html, 'store': <StoreBackendEntrypoint>
  Open AbstractDataStore instances in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.StoreBackendEntrypoint.html}


In [8]:
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})
BUCKET = "esam"

# Puffin data 
colonies_path = f"{BUCKET}/diffusion/puffin_data/puffin-data1_Colonies.csv"
with fs.open(colonies_path, "r") as f:  
        colonies_data = pd.read_csv(f)

data_path = f'{BUCKET}/diffusion/puffin_data/puffin-data1_Colonies.csv'
with fs.open(data_path, "r") as f:  
        puffin_data = pd.read_csv(f)

# Copernicus data 
dataframes = []
files_in_folder = fs.ls(f'{BUCKET}/diffusion/cds_data') 


file_path = "esam/diffusion/cds_data/tos_Omon_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_19500116-20141216.nc"


local_file = ".data/temp_file.nc"
with fs.open(file_path, "rb") as f:
    with open(local_file, "wb") as temp:
        temp.write(f.read())

# Lecture avec xarray
dataset = xr.open_dataset(local_file, engine="netcdf4")
print(dataset)


FileNotFoundError: [Errno 2] No such file or directory: '.data/temp_file.nc'

Remark : netCDF4 files or how to pull out your hair


In [ ]:
#for file_path in files_in_folder:
        with fs.open(file_path, "r") as f:
                dataset = xr.open_dataset(f, engine="netcdf4")  # Charger le fichier NetCDF
                print(dataset)
                df = dataset.to_dataframe().reset_index()
                dataframes.append(df) 


if dataframes:
    print(f"Extracted {len(dataframes)} DataFrames:")
    for i, df in enumerate(dataframes):
        print(f"DataFrame {i+1}: shape={df.shape}, columns={df.columns.tolist()}")
else:
    print("No DataFrames extracted.")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# 1. Charger les données
data = pd.read_csv('puffins_data.csv')

# 2. Définir les variables explicatives et cible
X = data[['Latitude', 'Longitude', 'SST_Janvier', 'SST_Juin', 'Année']]
y = data['Observations_Puffins']

# 3. Diviser les données en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Initialiser et entraîner le modèle
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 5. Prédire et évaluer le modèle
y_pred = rf_model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse:.2f}')
print(f'R²: {r2:.2f}')

# 6. Analyser l'importance des variables
importances = rf_model.feature_importances_
features = X.columns

plt.figure(figsize=(8, 6))
plt.barh(features, importances)
plt.xlabel('Importance')
plt.ylabel('Variables')
plt.title('Importance des variables dans le modèle Random Forest')
plt.show()

# 7. Prédictions futures
new_data = pd.DataFrame({
    'Latitude': [65.0, 66.5],
    'Longitude': [-18.0, -17.5],
    'SST_Janvier': [5.2, 4.8],
    'SST_Juin': [8.1, 7.9],
    'Année': [2015, 2016]
})

predictions = rf_model.predict(new_data)
print(f'Prédictions : {predictions}')
